In [1]:
import torch
from transformers import AutoModelForSeq2SeqLM

base_model_id = 'google/flan-t5-xl'
base_model = AutoModelForSeq2SeqLM.from_pretrained(
  base_model_id,
  torch_dtype=torch.bfloat16,
  load_in_8bit=True,
  device_map='auto'
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [2]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model_id)

In [3]:
from peft import PeftModel
model = PeftModel.from_pretrained(base_model, 'textminr/ner-flan-t5-xl')

adapter_config.json:   0%|          | 0.00/545 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/144M [00:00<?, ?B/s]

In [25]:
prompt_template = 'Input sentence: {words} --- Output:'
# sentence = 'A comprehensive research paper from Benjamin Kissinger details seafaring advancements, published in 1605, is a very good analysis.'
# sentence = "In a comprehensive research paper titled 'The Future of Renewable Energy: Trends and Predictions', published in 2025 by Benjamin Kissinger, an in-depth analysis of emerging technologies in the field of sustainable energy is presented. This detailed study examines the latest advancements in solar, wind, and geothermal energy, and evaluates their potential impact on global energy consumption patterns. The paper also explores the economic and environmental implications of transitioning to renewable energy sources, including the challenges and opportunities for developing countries. Despite the exhaustive nature of the research and its implications for future energy policies, ...."
# sentence = "In her comprehensive and deeply researched historical volume titled 'Empires in the Sands: The Ebb and Flow of Ancient Desert Civilizations', Dr. Helena Strauss, a renowned archaeologist and historian, delves into the rise and fall of civilizations that thrived in the harsh environments of the world's great deserts. This book, published in the spring of 2024, represents a culmination of over a decade of field research and scholarly analysis. Dr. Strauss meticulously reconstructs the socio-political and economic landscapes of these ancient societies, drawing from a vast array of sources, including archaeological excavations, ancient manuscripts, and cutting-edge remote sensing technologies. The first section of the book explores the ingenious adaptations and survival strategies developed by these civilizations to overcome the challenges posed by their arid surroundings, highlighting their agricultural innovations, water management techniques, and architectural marvels. The middle chapters provide an in-depth analysis of the trade networks established across the deserts, emphasizing how these routes facilitated not just the exchange of goods, but also the spread of ideas, religions, and cultures, profoundly influencing the broader course of human history. Dr. Strauss further examines the role of leadership and governance in these societies, from the centralized bureaucracies of the Egyptian pharaohs to the tribal confederations of the Berbers. In the latter part of the book, the focus shifts to the artistic and cultural achievements of these civilizations, showcasing their contributions to art, literature, and science, and how these continue to resonate in the modern world. The book concludes with a reflective discourse on the lessons that contemporary societies can learn from the successes and failures of these ancient desert civilizations, especially in the context of current environmental challenges and the looming threat of climate change. Dr. Strauss' work is not only a testament to the resilience and ingenuity of human societies in the face of adversity but also serves as a crucial reminder of the importance of preserving our cultural heritage and learning from the past."
# sentence = "In a comprehensive historical analysis titled 'Shadows and Light: The Dualistic Nature of the Industrial Revolution', a detailed exploration of the societal and technological transformations that occurred during this pivotal era is presented. The author, renowned for their expertise in European history, meticulously dissects the myriad ways in which the Industrial Revolution, spanning across the 19th century, reshaped the social fabric of the continent. This extensive work delves into the dichotomy of progress and disparity brought about by industrialization, examining how the advent of new machinery and mass production techniques led to unprecedented economic growth while simultaneously exacerbating class divisions and labor struggles. The study encompasses a wide array of topics, including the evolution of urban centers, the impact of the railway system on trade and mobility, and the profound changes in everyday life brought about by technological advancements. Additionally, the author scrutinizes the cultural and artistic responses to these changes, highlighting how literature, art, and philosophy of the time reflected the complex interplay between humanity and the mechanized world. The book also includes a critical analysis of the environmental consequences of industrial practices, shedding light on the early roots of the environmental movement. Despite its depth and breadth, the author's name is not mentioned anywhere in the manuscript, leaving their identity a subject of speculation among historians."
# sentence = "In a comprehensive research paper titled 'The Future of Renewable Energy: Trends and Predictions', published in 2025, an in-depth analysis of emerging technologies in the field of sustainable energy is presented. This detailed study examines the latest advancements in solar, wind, and geothermal energy, and evaluates their potential impact on global energy consumption patterns. The paper also explores the economic and environmental implications of transitioning to renewable energy sources, including the challenges and opportunities for developing countries. Despite the exhaustive nature of the research and its implications for future energy policies, the author's name is not mentioned in the document."
# sentence = "Berlin's Architecture', by Hans Becker in the 1800s, delves into postmodern design."
sentence = "The latest exhibition curated by Kelly Johnson, titled 'Evolving Paradigms: The Journey of Modern Education'. presents a fascinating exploration of contemporary educational systems, showcased in midwinter 2020. This exhibition, hosted at the renowned Global Education Center, delves into the transformation of educational methodologies from traditional to modern approaches. Johnson's work meticulously examines the shift from rote memorization to interactive and digital learning, emphasizing the impact of technological advancements on educational practices. The exhibition features interactive displays, historical educational artifacts, and a series of lectures and workshops led by prominent educators and scholars. It also critically analyzes the challenges and successes of integrating technology in classrooms across diverse socio-economic backgrounds. Furthermore, Johnson provides a comparative analysis of educational systems worldwide, highlighting innovative practices in various countries. She discusses the role of cultural influences in shaping educational policies and the growing importance of personalized learning experiences. The exhibition culminates in a forward-looking section, envisioning the future of education in an increasingly interconnected and digital world, with a focus on the potential of AI and VR in revolutionizing learning experiences."

In [26]:
prompt = prompt_template.format(words=sentence)

In [27]:
model_input = tokenizer(prompt, return_tensors="pt").to("cuda")

model.eval()
with torch.no_grad():
    print(tokenizer.decode(model.generate(**model_input, max_new_tokens=50)[0], skip_special_tokens=True))

/home/asuender/miniconda3/envs/ds/lib/python3.11/site-packages/bitsandbytes/autograd/_functions.py:322: UserWarning: MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


"author": "Kelly Johnson", "date": "2020"
